### 1. Introduction 

한국어 텍스트 분석을 하기 위해서는 문장들을 단어로 인식해야 합니다. 문장들을 단어열로 분해한다면, 우리는 각 문서에 어떤 단어가 몇 번 등장하였는지 살펴봄으로써 문서를 단어 빈도 행렬 (term freuquency matrix)로 표현할 수 있습니다. 혹은 Doc2Vec, Recurrent Neural Network (RNN) 이나 Convolutional Neural Network (CNN) 과 같은 뉴럴 네트워크 모델을 이용한 문서 표현을 할 수도 있습니다. 문장을 단어열로 분해하는 과정은 머신러닝 알고리즘에 한국어 텍스트 데이터를 입력하기 위한 기초 과정입니다.

데이터 분석의 목적과 사용할 알고리즘에 따라서 단어열의 정의가 달라질 수 있습니다. 딥러닝 모델을 이용하는 최근의 번역 엔진들은 단어를 정확히 인식하지 않기도 합니다. 번역기의 목적은 한 언어로 기술된 문장을 다른 언어로 번역하는 것이지, 문장 내의 단어를 정확히 인식하는 것은 아니기 때문입니다. 하지만 키워드를 추출하거나 토픽 모델링을 수행할 때에는 단어를 명확히 인식해야 할 때가 있습니다. 때로는 단어의 품사까지 파악해야 하는 경우도 있습니다. 분석의 목적에 따라서 단어열의 정의는 달라질 수 있습니다. 

우리는 아래의 예시처럼 주어진 문장에서 명사, 동사, 혹은 조사와 같은 단어들로 구분하는 품사 판별기를 만들어 보겠습니다. 앞서 언급한 것처럼, 공개되어 있는 품사 판별기를 그대로 이용하면 기대했던 것과 다른 단어열의 결과가 발생하기도 합니다. 우리는 각자의 분석에 맞게 알고리즘을 고쳐 쓸 필요가 있습니다. 혹은 분석하려는 데이터의 특성에 맞게 알고리즘을 새로 만들어야 하는 일들도 빈번합니다. 이를 위해서 품사 판별기의 원리를 알아보고 나의 품사 판별기를 직접 만들어 봄으로써, 좀 더 좋은 데이터 분석의 결과물을 만들어 봅시다. 

우리는 아래의 예시처럼 주어진 문장을 단어열로 인식하는 간단한 품사 판별기를 만들어 보겠습니다. 품사 판별기의 세부 구성 요소는 크게 단어열 후보를 만드는 과정과 생성된 후보 중에서 가장 적절한 것을 선택하는 스코어링 과정으로 구성됩니다. 

품사 판별기는 문장이 주어지면 가능한 단어열 후보를 만듭니다. 아래의 예시 문장에서 '너무' 는 '부사이기도 하기 때문에 어절 '너무너무너무는' 은 '너무 + 너무 + 너무 + 는'으로 인식될 수도 있습니다. 만약 단어열 후보가 한 가지라면 명확하겠지만, 여러 개의 단어열 후보가 만들어진다면 그 중에서 가장 적절한 후보를 선택해야 합니다. 품사 판별기의 두번째 구성요소는 여러 개의 단어열 후보 중에서 가장 적절한 후보를 최종 결과로 선정하는 스코어링 과정입니다. 이는 뒤에서 좀 더 다뤄봅니다. 

    문장: '너무너무너무는 아이오아이의 노래입니다'
    단어열: [('너무너무너무', '명사'), ('는', '조사'), ('아이오아이', 명사'), ('의', '조사'), ('노래', '명사'), ('입니다', '조사')]
    
단어열 후보를 만들기 위해서는 단어 사전이나 규칙이 이용될 수 있습니다. 아래의 사전을 이용한다면 어절 '너무너무너무는'은 '너무너무너무/명사 + 는/조사'와 '너무/부사 + 너무/부사 + 너무/부사 + 는/조사'로 인식될 수 있습니다. 혹은 '-은', -'는', '-이', '-가', '-나'와 같이 자주 이용되는 조사 앞에 등장하는 글자는 명사라는 규칙을 이용할 수도 있습니다. 하지만 규칙을 이용하는 것은 때론 잘못된 후보를 만들 가능성이 있습니다. 구어체에서는 조사가 쓰이지 않는 경우가 많습니다 '-나' 앞의 글자를 명사 후보로 만든다면, '강미나 팬클럽'이라는 문장은 '강미/명사 + 나/조사 + 팬클럽/명사'과 '강미나/명사 + 팬클럽/명사'의 후보가 만들어 질 수 있습니다. 적절하지 않은 단어열 후보는 스코어링 과정에서 걸러낼 수 있습니다. 

    단어 사전 = {
               '부사': {'너무', '매우'}, 
               '명사': {'너무너무너무', '아이오아이', '아이', '노래', '오'},
               '조사': {'는', '의', '이다', '입니다'}
              }

스코어링은 여러 개의 단어열 후보 중에서 문장에 가장 적합한 단어열을 선택하는 과정입니다. 위 예문에서 아래와 같은 문장에 대한 단어열 후보를 만들 수 있습니다. 이들의 점수를 계산한 뒤, 높은 점수를 보이는 후보를 최종 단어열로 선정합니다. 스코어링은 한 문장에 대한 단어열에 대하여 이뤄질 수도 있지만, 문장 내의 어절 단위로 이뤄질 수도 있습니다. 

    문장 : 너무너무너무는 아이오아이의 노래입니다
    후보 1: [너무너무너무/명사, 는/조사, 아이오아이/명사, 의/조사, 노래/명사, 입니다/조사]
    후보 2: [너무너무너무/명사, 는/조사, 아이/명사, 오/명사, 아이/명사, 의/조사, 노래/명사, 입니다/조사]

이 방법은 인접한 어절들의 문맥을 모두 고려하기 때문에 모호성이 줄어들지만, 여러 개의 단어열 후보가 만들어질 수 있습니다. 반대로 아래의 예시처럼 어절 단위로 단어열 후보의 스코어링을 수행할 수도 있습니다. 우리는 어절 '아이오아이의'의 두 후보 중에서 점수가 높은 단어열을 어절의 최종 단어열로 선정한 뒤, 다음 어절 '노래입니다'의 단어열은 을 선택할 수 있습니다. 품사 판별기의 직관적인 이해를 위해서 우리는 먼저 어절 단위로 스코어링을 수행하는 품사 판별기를 만들어 봅니다 (사실, 앞/뒤의 문맥을 모두 봐야 모호성이 해결되는 경우는 그렇게 많지 않습니다).

    어절: 아이오아이의 
    후보 1: [아이오아이/명사, 의/조사]
    후보 2: [아이/명사, 오/명사, 아이/명사, 의/조사]

## 2. 품사 사전 클래스

우리는 품사를 알고 있는 단어 사전으로부터 주어진 문장의 단어열을 분석하는, "사전 기반 품사 판별기"를 만듭니다. 단어열의 후보는 단어 사전에 존재하는 단어들을 string match 하여 만듭니다. 우리는 이후에 품사 단어 사전을 저장하거나 파일에서 읽어오기도 할 것이며, 단어열 후보를 만드는데도 이용할 것입니다. 미리 class 형식으로 만들어둡니다. 지금은 품사 판별기를 만들기 위한 테스트용 단어 사전을 만듭니다. 

품사 단어 사전은 다음의 함수로 구성되어 있습니다. 

    __init__()
    _check_max_length(pos_dict)
    get_pos(word)
    word_is_tag(word, tag)

\_check\_max\_length(pos\_dict) 함수는 현재 사전에 등록된 단어 중에서 가장 긴 단어의 길이를 찾는 함수입니다. list comprehension 과 같은 멋진 파이썬의 문법을 이용하면 손쉽게 계산할 수 있습니다. 함수 이름의 맨 앞에 \_를 넣어 함수를 숨깁니다. 사용자가 반드시 알아야 하는 함수는 아니기 때문입니다.

get_pos(word) 함수는 단어 word 를 입력받은 뒤, 사전에 존재하는 모든 품사를 모아 tags 라는 list 로 return 합니다. 

word_is_tag(word, tag) 는 주어진 단어 word 가 주어진 품사 tag 인지 확인하는 함수입니다. 

In [1]:
from pprint import pprint

class Dictionary:
    def __init__(self, pos_dict=None):
        self.pos_dict = {
            '부사': {'너무', '매우'}, 
            '명사': {'너무너무너무', '아이오아이', '아이', '노래', '오', '이', '고양'},
            '조사': {'는', '의', '이다', '입니다', '이', '이는', '를', '라', '라는'},
            '동사': {'하는', '하다', '하고'},
            '형용사': {'예쁜', '예쁘다'},
            '감탄사': {'우와'}
          }
        self.max_length = self._check_max_length(self.pos_dict)
    
    def _check_max_length(self, pos_dict):
        return max((len(word) for words in pos_dict.values() for word in words))
    
    def get_pos(self, word):
        tags = []
        for pos, words in self.pos_dict.items():
            if word in words:
                tags.append(pos)
        return tags
    
    def word_is_tag(self, word, tag):
        return word in self.pos_dict.get(tag, {})

dictionary = Dictionary()

dictionary 를 만든 뒤, pprint 를 이용하여 저장이 되어 있는 단어들을 확인해 봅니다. 

In [2]:
pprint(dictionary.pos_dict)

{'감탄사': {'우와'},
 '동사': {'하는', '하고', '하다'},
 '명사': {'아이', '고양', '아이오아이', '오', '이', '너무너무너무', '노래'},
 '부사': {'매우', '너무'},
 '조사': {'라는', '이다', '를', '는', '입니다', '이는', '의', '라', '이'},
 '형용사': {'예쁜', '예쁘다'}}


get_pos() 함수에 '아이오아이'를 입력하면 명사만 출력됩니다. 

In [3]:
dictionary.get_pos('아이오아이')

['명사']

하지만 '이'를 입력하면 '조사'와 '명사'가 모두 출력됩니다. 

In [4]:
dictionary.get_pos('이')

['명사', '조사']

'아이오아이/명사'가 사전에 등록되어 있는지 확인할 수도 있습니다. 

In [5]:
dictionary.word_is_tag('아이오아이', '명사')

True

'너무너무너무'의 길이가 6 이기 때문에 dictionary.max_length 는 6 입니다. 우리는 이를 이용하여 효율적으로 단어 후보를 생성할 것입니다. 이제 우리가 만든 테스트용 dictioanry 를 이용하여 문장 '너무너무너무는 아이오아이의 노래입니다'의 단어 후보를 만들어 봅시다. 

In [6]:
dictionary.max_length

6

## 3.  띄어쓰기가 잘 지켜진 데이터셋

### 3.1. 후보 생성

어절 '너무너무너무는'에 대하여 단어의 사전 등록 유무를 무시한 체 모든 단어열 후보를 생성하면 아래와 같이 $2^7-1$ 개의 후보가 만들어집니다. 이는 우리가 원하는 것이 아닙니다. 아무리 스코어링 과정에서 옳지 않은 단어열 후보를 걸러낼 수 있다 하더라도, 애초에 단어열이 될 가능성이 적은 후보들을 만들 필요는 없습니다. 

	'너무너무너무는'
	'너무너무너무 + 는'
	...
	'너 + 무 + 너 + 무 .. + 는' 

우리는 한국어의 어절 구조의 특징을 이용하여 단어열 후보를 생성해 봅시다. 앞서 살펴본 것처럼 한국어의 품사는 5언 9품사입니다. 이 중에서도 의미를 지니는 품사는 명사/동사/형용사/부사/감탄사 입니다. 어문학적인 분석이 아닌 키워드 추출이나 토픽 모델링과 같은 데이터 분석을 하기 위해서는 이 다섯가지 단어들을 잘 인식하는 것이 더 중요하기도 합니다. 그리고 이 단어들은 어절의 왼쪽에 존재합니다. 만약 띄어쓰기가 잘 지켜져 있다면 띄어쓰기 기준으로 어절을 구분한 뒤, 각 어절에서 가능한 단어열을 후보를 생성할 수 있습니다. 

또한 후보 생성에 몇 가지 제약들도 있습니다. 조사는 명사 다음에 등장합니다. '조사 + 조사'의 단어열은 문법적으로 옳지 않은 단어열입니다. 한 어절을 구성할 수 있는 품사 형태가 정의되어 있다면 탬플릿 매칭을 할 수도 있습니다. 명사/형용사/동사/부사/감탄사는 그 자체로 하나의 어절을 이룰 수 있습니다. 혹은 '명사 + 조사'의 형태로 어절이 구성될 수 있습니다. 하지만 가끔 '명사 + 동사' 혹은 '명사 + 형용사' 처럼 보이는 어절이 존재합니다. 예를 들어 '노래하는' 이라는 어절은 '노래/명사 + 하는/동사'로 보이기도 합니다. 이 어절은 본래 '노래/명사 + 하/동사형전성어미 + 는/어말어미' 로 이뤄져 있습니다. 전성어미는 품사의 종류를 바꿔준다는 의미입니다. 명사 뒤에 전성어미가 결합되어 '노래하다'나 '모델같다'와 같은 동사/형용사가 만들어 집니다. 하지만 '하다', '하는', 같다', '같은'은 모두 동사와 형용사이니 명사 뒤에 동사나 형용사가 올 수도 있다는 템플릿을 만들어도 좋을 것 같습니다. 

물론 '노래하다', '노래하는'을 모두 동사 사전에 추가할 수도 있습니다만, 명사와 동사를 구분하여 사전에 입력한다면 '노래 + 같은'과 같이 사전에 등장하지 않은 어절에 대해서도 인식할 수 있습니다. 지금까지의 이야기는 띄어쓰기가 잘 지켜져 있다고 가정하기 때문에 우리가 고려할 탬플릿은 그리 많지 않습니다. 즉 우리는 띄어쓰기가 잘 되어 있을 때 다음과 같은 탬플릿을 만들 수 있습니다. 

    templates = [('명사', ), ('동사', ), ('형용사', ), ('부사', ), ('감탄사', ), 
                 ('명사', '동사'), ('명사', '형용사'), ('명사', '조사')]

그러나 우리가 반드시 사전이 잘 구축되어 있다고 장담할 수도 없습니다. '고양이'는 '고양/명사 + 이/조사'로 인식될 수도 있지만, 그 자체로 사전에 없는 단어로 인식되야 하기도 합니다. 사전이 완벽하지 않음에도 사전에 존재하는 단어로만 단어열 후보를 만들게 되면 '아이오아이'라는 단어를 모를 때, '아이 + 오 + 아이' 처럼 알려진 단어들로 잘못 분해할 수 있기 때문입니다. 우리는 어절 자체가 알려진 단어가 아닌 경우에는 None 으로 품사 태그를 달아두도록 합시다. 

혹은 함 품사의 단어가 여러 번 반복되기도 합니다. '저녁'과 '약속'은 단어 사전에 등록되어 있지만, '저녁약속'이 등록되어 있지 않을 수도 있습니다. 혹은 부사가 강조의 의미로 여러 번 반복되기도 합니다. '너무너무너무 보고싶어' 라는 문장의 첫번째 어절은 '너무/부사'가 3 번 반복된 어절이기도 합니다. 

그렇다면 우리는 위 템플릿을 이용하여 띄어쓰기가 잘 되어 있는 문장에서 단어열 후보를 만들 수 있습니다. 

In [7]:
from collections import namedtuple

# b:begin, m: middle, e: end
LR = namedtuple('LR', 'l l_tag r r_tag b m e')

print(LR('아이오아이', '명사', '', None, 0, 5, 5))
print(LR('아이오아이', '명사', '는', '조사', 0, 5, 6))

LR(l='아이오아이', l_tag='명사', r='', r_tag=None, b=0, m=5, e=5)
LR(l='아이오아이', l_tag='명사', r='는', r_tag='조사', b=0, m=5, e=6)


In [8]:
class BaseTemplateMatcher:
    def generate(self, token):
        raise NotImplementedError

class EojeolTemplateMatcher(BaseTemplateMatcher):
    def __init__(self, dictionary, single_tags=None, lr_templates=None):
        if not single_tags:
            single_tags = ['명사', '동사', '형용사', '부사', '감탄사']
        if not lr_templates:
            lr_templates = [('명사', '동사'), ('명사', '형용사'), ('명사', '조사')]
        self.dictionary = dictionary
        self.single_tags = single_tags
        self.lr_templates = lr_templates

    def generate(self, eojeol):
        n = len(eojeol)
        candidates = [LR(eojeol, None, '', None, 0, n, n)]
        for tag in self.dictionary.get_pos(eojeol):
            if tag in self.single_tags:
                candidates.append([LR(eojeol, tag, '', None, 0, n, n)])

        for b in range(1, n):
            l, r = eojeol[:b], eojeol[b:]
            for l_tag, r_tag in self.lr_templates:
                if self.dictionary.word_is_tag(l, l_tag) and self.dictionary.word_is_tag(r, r_tag):
                    candidates.append([LR(l, l_tag, r, r_tag, 0, b, n)])
        
        compound_noun = self._decompose_compound(eojeol, '명사')
        if compound_noun:
            candidates.append(compound_noun)
            
        compound_adverb = self._decompose_compound(eojeol, '부사')
        if compound_adverb:
            candidates.append(compound_adverb)

        return candidates

    def _decompose_compound(self, eojeol, tag):
        n, b = len(eojeol), 0
        words = []
        while b < n:
            e = b + 1
            next_round = False
            for i in range(e, n+1):
                if next_round or (b == 0 and i == n):
                    break
                subword = eojeol[b:i]
                if self.dictionary.word_is_tag(subword, tag):
                    words.append(LR(subword, tag, '', None, b, i, i))
                    b = i
                    next_round = True
                    break
            if not next_round:
                return []
        return words

generator = EojeolTemplateMatcher(dictionary)

사전에는 '아이', '오', '아이오아이'라는 명사가 포함되어 있습니다. 어절 '아이오아이'는 두 개의 단어열 후보가 만들어집니다. 

In [9]:
for i, candidate in enumerate(generator.generate('아이오아이')):
    print('\ncandidate #{}'.format(i))
    pprint(candidate)


candidate #0
LR(l='아이오아이', l_tag=None, r='', r_tag=None, b=0, m=5, e=5)

candidate #1
[LR(l='아이오아이', l_tag='명사', r='', r_tag=None, b=0, m=5, e=5)]

candidate #2
[LR(l='아이', l_tag='명사', r='', r_tag=None, b=0, m=2, e=2),
 LR(l='오', l_tag='명사', r='', r_tag=None, b=2, m=3, e=3),
 LR(l='아이', l_tag='명사', r='', r_tag=None, b=3, m=5, e=5)]


'는' 이라는 조사가 있기 때문에 '아이오아이/명사 + 는/조사' 후보가 만들어 집니다.

In [10]:
for i, candidate in enumerate(generator.generate('아이오아이는')):
    print('\ncandidate #{}'.format(i))
    pprint(candidate)


candidate #0
LR(l='아이오아이는', l_tag=None, r='', r_tag=None, b=0, m=6, e=6)

candidate #1
[LR(l='아이오아이', l_tag='명사', r='는', r_tag='조사', b=0, m=5, e=6)]


사전에 '너무너무너무' 라는 명사와 '너무'라는 부사가 존재합니다. '너무/부사'가 세 번 연속 반복되는 후보도 만들어 집니다. 

In [11]:
for i, candidate in enumerate(generator.generate('너무너무너무')):
    print('\ncandidate #{}'.format(i))
    pprint(candidate)


candidate #0
LR(l='너무너무너무', l_tag=None, r='', r_tag=None, b=0, m=6, e=6)

candidate #1
[LR(l='너무너무너무', l_tag='명사', r='', r_tag=None, b=0, m=6, e=6)]

candidate #2
[LR(l='너무', l_tag='부사', r='', r_tag=None, b=0, m=2, e=2),
 LR(l='너무', l_tag='부사', r='', r_tag=None, b=2, m=4, e=4),
 LR(l='너무', l_tag='부사', r='', r_tag=None, b=4, m=6, e=6)]


### 3.2. 스코어링

여러 개의 단어열 후보 중에서 가장 적절한 후보를 선택하기 위하여 스코어링을 합니다. '너무너무너무'는 '명사', '미등록단어', '부사 + 부사 + 부사'의 세 가지 후보가 존재하는데, 이중에서 가장 적절한 단어열을 선택하는 것입니다. 

이를 위하여 여러가지 머신러닝 기법들이 이용될 수 있습니다. Logistic regression 의 확장인 Conditional Random Field (CRF)는 이를 위해 많은 품사 판별기 / 형태소 분석기에서 이용이 되었습니다. 그 이전에는 Hidden Markov Model (HMM) 도 같은 목적을 위하여 이용이 되었습니다. 그러나 이 머신러닝 알고리즘들은 학습데이터가 필요합니다. 이들은 학습데이터를 이용하여 P(너무너무너무/명사), P(너무/부사 + 너무/부사 + 너무/부사) 의 확률값을 학습합니다. 하지만 학습데이터를 이용하는 방법들은 두 가지 문제가 발생합니다. 첫째로 대부분의 경우 우리는 학습데이터를 가지고 있지 않습니다. P(너무너무너무/명사)의 확률은 학습데이터에 이 단어가 존재해야 학습이 됩니다. 그러나 새롭게 만들어지는 단어들을 모두 포함하는 학습데이터가 존재할리 없습니다. 두번째, 학습데이터의 확률 기준에 맞춰 스코어링을 수행하는 것은 학습데이터의 관점으로 다른 데이터를 제단한다는 의미이기도 합니다. 우리가 2016 년의 연예뉴스기사를 분석하기 위하여 2010 년에 작성된 학습데이터를 이용한다면, 여러 개의 단어열 후보가 발생했을 때, 2016 년의 표현을 선택하지 않고, 2010 년도의 표현을 선택하게 됩니다. 그렇다면 오히려 우리가 분석하려는 데이터에 맞게 그 경향을 조절할 수 있도록 규칙을 이용하여 스코어링을 하는 것이 현실적인 방법일 수도 있습니다. 

예를 들어, 복합명사가 그 자체로 사전에 등록되어 있을 때, 각 명사를 분해하는 것보다 복합명사 자체를 그대로 인식하기를 원한다면 단어열 후보 내의 명사 개수가 많을수록 음의 점수를 부여할 수 있습니다. 한 단어가 명사/부사 모두 후보가 존재할 때 명사를 선호한다면, 부사보다 명사인 단어에 양의 점수를 더해줄 수도 있습니다. 사실 이는 Logistic regression 의 방식이기도 합니다. Features 를 만든 뒤, coefficients 를 통하여 각 feature 에 점수를 더해주는 것입니다. 기존에 품사 판별기에서 많이 이용되던 Conditional Random Field 도 Logistic Regression 계열의 모델입니다. 머신러닝 알고리즘이 학습할 정보에 대하여 우리가 이미 알고있는, 혹은 선호하는 지식이 있다면 이를 그대로 이식하는 것입니다. 

데이터분석을 수행할 때 많은 경우 정답 혹은 학습데이터가 존재하지 않는 경우가 많습니다. 이 때에는 이미 알려진 지식을 활용해도 충분히 좋은 성과를 얻을 수 있습니다. 

Evaluator 는 여러 단어열 후보들 중에서 가장 적절한 후보를 선택하는 클래스입니다. 각자의 문제에 맞게 변형할 수 있도록 BaseEvaluator 를 상속받도록 하면 코드 관리가 용이합니다. Evaluator 는 하나의 단어열 후보 (candidate) 에 대하여 점수를 부여하는 evaluate() 와 여러 개의 단어열 후보들 중에서 가장 높은 점수를 받은 후보를 선택하는 select_best() 로 구성하였습니다. 

아래의 SimpleEojeolEvaluator 는 여덞개의 평가 항목을 이용하여 단어열 후보 점수를 계산합니다. 그 중 명사의 개수가 많을수록 그 개수에 비례하여 0.1 만큼 점수를 내렸습니다. 이는 복합명사를 여러 개의 단일 명사보다 더 선호하겠다는 의미입니다. 그리고 이 부분은 evaluate() 에서 아래와 같은 구문으로 구현되어 있습니다. 

	num_nouns = len([True for lr in candidate if lr.l_tag == '명사'])

그리고 이 평가 항목에 weight 의 첫번째 항목을 가져옴으로써 점수를 매길 수 있습니다. 

	score += num_nouns * self.weights[0][1]

다른 항목들도 이와 비슷하게 만들 수 있습니다. 반드시 5언 9품사의 구조가 아닌, 자신의 문제에 적합한 태그셋이 있다면, 그에 맞춰서 평가 항목을 변형할 수도 있습니다. 

In [12]:
class BaseEvaluator:
    def evaluate(self, candidate):
        # Develop your evaluation functions
        return 0
    def select_best(self, candidates):
        if not candidates:
            return None
        scored = [(c, self.evaluate(c)) for c in candidates]
        best = sorted(scored, key=lambda x:-x[1])[0][0]
        return best

class SimpleEojeolEvaluator(BaseEvaluator):
    def __init__(self, weights=None):
        if not weights:
            weights = (
                ('num_nouns', -0.1),
                ('num_words', -0.15),
                ('has_noun', 0.2),
                ('has_unknown', -0.3),
                ('is_adverb_repeat', 0.1),
                ('is_compound_noun', -0.05),
                ('is_noun_josa', 0.1),
                ('is_a_noun', 0.2)
            )
        self.weights = weights
        
    def evaluate(self, candidate):
        if isinstance(candidate, LR):
            candidate = [candidate]
        num_nouns = len([True for lr in candidate if lr.l_tag == '명사'])
        num_words = len(candidate)
        has_noun = num_nouns > 0
        has_unknown = len([True for lr in candidate if lr.l_tag is None])
        is_adverb_repeat = (num_words > 1 and 
            num_words == len([True for lr in candidate if lr.l_tag == '부사']))
        is_compound_noun = (num_words > 1 and
            num_words == len([True for lr in candidate if lr.l_tag == '명사']))
        is_noun_josa = (num_words == 1 and
            (candidate[0].l_tag == '명사' and candidate[0].r_tag == '조사'))
        is_a_noun = num_words == 1 and candidate[0].l_tag == '명사' and not candidate[0].r
        
        score = (num_nouns * self.weights[0][1]
                 + num_words * self.weights[1][1]
                 + has_noun * self.weights[2][1]
                 + has_unknown * self.weights[3][1]
                 + is_adverb_repeat * self.weights[4][1]
                 + is_compound_noun * self.weights[5][1]
                 + is_noun_josa * self.weights[6][1]
                 + is_a_noun * self.weights[7][1])

        return score

In [13]:
evaluator = SimpleEojeolEvaluator()

candidates = generator.generate('아이오아이는')
pprint(candidates)

[LR(l='아이오아이는', l_tag=None, r='', r_tag=None, b=0, m=6, e=6),
 [LR(l='아이오아이', l_tag='명사', r='는', r_tag='조사', b=0, m=5, e=6)]]


In [14]:
for candidate in candidates:
    print('candidate: %s' % str(candidate))
    print('score: %.3f\n' % evaluator.evaluate(candidate))

candidate: LR(l='아이오아이는', l_tag=None, r='', r_tag=None, b=0, m=6, e=6)
score: -0.450

candidate: [LR(l='아이오아이', l_tag='명사', r='는', r_tag='조사', b=0, m=5, e=6)]
score: 0.050



In [15]:
evaluator.select_best(candidates)

[LR(l='아이오아이', l_tag='명사', r='는', r_tag='조사', b=0, m=5, e=6)]

In [16]:
def tag(sentence, flatten=True):
    sent_ = []
    eojeols = sentence.split()
    for eojeol in eojeols:
        candidates = generator.generate(eojeol)
        words = evaluator.select_best(candidates)
        sent_.append(flatten_words(words))
    
    if flatten:
        sent_ = [word for words in sent_ for word in words]
    return sent_

def flatten_words(words):
    words_ = []
    for word in words:
        if word.l:
            words_.append((word.l, word.l_tag))
        if word.r:
            words_.append((word.r, word.r_tag))
    return words_

띄어쓰기가 잘 지켜져 있다면, 각 어절은 띄어쓰기로 구분이 됩니다. 한 문장에 대한 품사 판별은 먼저 띄어쓰기 기준으로 어절을 나눈 뒤, 각 어절마다 독립적으로 수행할 수 있습니다. 물론 각 어절에 대하여 독립적인 품사 판별을 진행할 경우, 모호성이 발생하여 제대로 작동하지 않는 경우도 있습니다. 하지만 그 경우는 적을 뿐더러, 이부분까지 고려할 수 있는 품사 판별기는 이후에 더 자세히 다뤄봅니다.

tag(sentence) 는 문장을 입력받은 뒤, 띄어쓰기로 어절을 구분합니다. 각 어절마다 단어열 후보를 만들고, 가장 적절한 후보를 선택하여 sent_ 에 누적합니다. 

	for eojeol in eojeols:
        candidates = generator.generate(eojeol)
        words = evaluator.select_best(candidates)
        sent_.append(flatten_words(words))

때로는 각 어절이 어떤 단어열로 나눠졌는지 구분을 해야 할 수도 있습니다. 혹은 한 문장에 대하여 하나의 단어열 리스트를 기대할 수도 있습니다. flatten 을 이용하면 두 경우의 품사 판별 결과를 손쉽게 만들 수 있습니다. 우리는 어절 단위로 단어열을 sent_ 에 저장하였습니다. flatten=True 일 경우에는 이를 그대로 return 합니다. 아마도 우리는 이를 더 많이 쓸테니 default value 로 flatten=True 를 설정해 둡니다. 그럼 아래와 같은 최종 결과를 얻을 수 있습니다. 

	sent = '너무너무너무는 아이오아이의 노래입니다'
	tag(sent)

	[('너무너무너무', '명사'),
	 ('는', '조사'),
	 ('아이오아이', '명사'),
	 ('의', '조사'),
	 ('노래', '명사'),
	 ('입니다', '조사')]


In [17]:
sent = '너무너무너무는 아이오아이의 노래입니다'
tag(sent)

[('너무너무너무', '명사'),
 ('는', '조사'),
 ('아이오아이', '명사'),
 ('의', '조사'),
 ('노래', '명사'),
 ('입니다', '조사')]

만약 각 어절마다 어떤 단어열이 만들어졌는지 확인해야 한다면 아래와 같이 함수를 실행시킵니다. 

	tag(sent, flatten=False)

	[[('너무너무너무', '명사'), ('는', '조사')],
 	[('아이오아이', '명사'), ('의', '조사')],
 	[('노래', '명사'), ('입니다', '조사')]]

In [18]:
tag(sent, flatten=False)

[[('너무너무너무', '명사'), ('는', '조사')],
 [('아이오아이', '명사'), ('의', '조사')],
 [('노래', '명사'), ('입니다', '조사')]]

우리는 tag(sentence, flatten) 와 to_words(words) 를 포함하는 SimpleTagger 를 만들었습니다. 품사 판별기 역시 각각의 문제에 적합한 여러 개의 변형 모델들을 만들 수 있습니다만, 단어열 후보를 만드는 과정과 이를 평가하는 과정의 구조는 유지하기 위해 BaseTagger 를 상속하도록 합니다. 여기에 후처리과정까지 수행할 수 있도록 postprocessor 도 입력합니다. 지금은 후처리는 하지 않고 있습니다. 


In [19]:
class BaseTagger:
    def __init__(self, generator, evaluator, postprocessor=None):
        self.evaluator = evaluator
        self.generator = generator
        self.dictionary = generator.dictionary
        self.postprocessor = postprocessor

    def tag(self, sentence, flatten=True):
        raise NotImplementedError

class BasePostprocessor:
    def postprocess(self, token, best_wordstream):
        return best_wordstream
        
class SimpleTagger(BaseTagger):
    def tag(self, sentence, flatten=True):
        sent_ = []
        eojeols = sentence.split()
        for eojeol in eojeols:
            candidates = generator.generate(eojeol)
            words = evaluator.select_best(candidates)
            
            words_ = []
            for word in words:
                if word.l:
                    words_.append((word.l, word.l_tag))
                if word.r:
                    words_.append((word.r, word.r_tag))

            sent_.append(words_)

        if flatten:
            sent_ = [word for words in sent_ for word in words]
        return sent_

In [20]:
tagger = SimpleTagger(generator, evaluator)
tagger.tag(sent)

[('너무너무너무', '명사'),
 ('는', '조사'),
 ('아이오아이', '명사'),
 ('의', '조사'),
 ('노래', '명사'),
 ('입니다', '조사')]

In [21]:
tagger.tag(sent, flatten=False)

[[('너무너무너무', '명사'), ('는', '조사')],
 [('아이오아이', '명사'), ('의', '조사')],
 [('노래', '명사'), ('입니다', '조사')]]

## 4. 띄어쓰기 오류가 많은 데이터

### 4.1. 후보 생성

어절의 구조인 L + [R]을 이용하여 어절의 왼쪽 부분의 substring 이 단어 사전에 등록되어 있는지만 확인하여도 사전 기반 품사 판별기를 만들 수 있습니다. 하지만 사람에 의하여 작성되는 한국어 텍스트에는 띄어쓰기 오류가 포함되는 경우가 많습니다. 특히 온라인 환경에서 작성되는 텍스트들은 띄어쓰기 오류가 빈번합니다. 물론 띄어쓰기 오류를 모두 제거한 뒤 품사 판별을 할 수도 있습니다만, 띄어쓰기 오류가 완벽히 제거된다는 가정을 하여야 합니다. 그보다도, 사람은 띄어쓰기가 잘 지켜지지 않은 한국어 문장들을 잘 이해합니다. 우리도 품사 판별기를 그렇게 만들면 됩니다. 우리의 목표는 아래의 띄어쓰기 오류를 포함하는 문장에 대하여 품사 판별을 수행하는 것입니다. 

	sent = '너무너무너무라는아이오아이의노래입니다'

우리가 만들 품사 판별기의 컨셉은, "아는 단어(어절) 부터 제대로 인식하자" 입니다. 위의 문장을 읽었을 때, 제 눈에는 '아이오아이' 라는 단어가 눈에 들어옵니다. 분석하는 문서 집합이 연예 뉴스라면 이는 매우 자연스러운 현상입니다. 우리가 데이터분석 기술 서적을 읽고 있는 중이라면, '로지스틱', '리그레션' 같은 단어들은 띄어쓰기가 잘 지켜지지 않더라도 눈에 띌 것입니다. 어떤 맥락에서 기대되는 단어들이 있기 때문에 우리는 아는 단어부터 눈에 들어옵니다. 그 뒤에는 '노래', '너무너무너무' 같은 단어가 눈에 들어옵니다. 이처럼 아는 단어부터 제대로 품사를 부여하는 품사 판별기를 만들 것입니다. 그리고 이 품사 판별기의 구조도 (1) 단어열 후보를 생성하는 generator, (2) 그 중에서 가장 적절한 단어열을 선택하는 evaluator 로 구성됩니다. 

이를 위해서 우리는 주어진 어절에서 단어열 후보를 만드는 과정을 네 가지 부분 과정으로 나누어 구현할 것입니다. 첫번째 과정은 단어 사전에 주어진 substring 이 의미를 지니는 단어인 명사/형용사/동사/부사/감탄사가 존재하는지 확인하는 과정 (initialize_L)입니다. 우리는 단어 사전을 이용하여 문장에서 이 다섯개 품사의 단어들이 존재하는지 확인할 것입니다. _pos_L(word) 는 단어가 이 다섯개 품사에 해당하는지 확인하는 함수입니다. String match 로 후보를 만든 뒤, 다음의 list 를 만듭니다. 

	[l, l_tag, b, e]

l 은 해당 단어, l_tag 는 그 단어의 품사, b, e 는 문장에서의 시작 위치 (b) 와 종료 위치 (e) 입니다. 한 단어에 대하여 여러 개의 품사가 존재할 수 있으니 우리는 아래처럼 모든 품사에 대하여 단어 후보를 만듭니다. 

	for l_tag in l_tags:
		candidates.append([l, l_tag, b, e])

후보들은 시작 위치 (b) 기준으로 정렬합니다. 

In [22]:
ltags = {'명사', '형용사', '동사', '부사', '감탄사'}

def _pos_L(word):
    poses = dictionary.get_pos(word)
    poses = {pos for pos in poses if pos in ltags}
    return poses

def _initialize_L(t):
    n = len(t)
    candidates = []

    for b in range(n):
        for e in range(b+2, min(n, b + dictionary.max_length) + 1):
            l = t[b:e]
            l_tags = _pos_L(l)

            if not l_tags:
                continue

            for l_tag in l_tags:
                candidates.append([l, l_tag, b, e])

    return sorted(candidates, key=lambda x:x[2])

['너무너무너무', '명사', 0, 6] 의 후보가 만들어지기도 하지만, ['너무', '부사', 0, 2] 와 같은 후보도 만들어집니다. 또한 '아이오아이' 에서는 '아이' 라는 단어들이 분해되어 후보로 만들어집니다. 만약 한 품사에 대하여 한 단어가 여러 개의 단어로 이뤄져 있다면, 최장일치법 (같은 품사 중에서는 가장 긴 단어를 선택)을 이용하여 후보를 줄입니다. 

	['아이', '명사', 8, 10],
	['아이오아이', '명사', 8, 13],
	['아이', '명사', 11, 13],

In [23]:
sent = '너무너무너무라는아이오아이의노래입니다'

_initialize_L(sent)

[['너무', '부사', 0, 2],
 ['너무너무너무', '명사', 0, 6],
 ['너무', '부사', 2, 4],
 ['너무', '부사', 4, 6],
 ['아이', '명사', 8, 10],
 ['아이오아이', '명사', 8, 13],
 ['아이', '명사', 11, 13],
 ['노래', '명사', 14, 16]]

\_remove\_subset\_l(candidates) 는 여러 개의 후보들 중에서 최장일치법으로 불필요한 후보를 제거하는 함수입니다. 단어의 길이 순으로 정렬한 후보들을 sorted\_ 에 저장합니다. 단어의 길이는 (x[3] - x[2]) 로 알 수 있습니다. 

	sorted_ = sorted(
            filter(lambda x:x[1] == l_tag, candidates), 
            key=lambda x:-(x[3] - x[2])
        )

가장 긴 단어를 선택하여 candidates\_ 에 append 한 뒤, 시작점과 끝점의 위치를 확인하여 겹치는 단어들을 sorted\_ 에서 제거합니다. 

    while sorted_:
        candidates_.append(sorted_.pop(0))
        (b, e) = (candidates_[-1][2], candidates_[-1][3])

         # Find overlapped
        removals = [i for i, c in enumerate(sorted_) if b <= c[2] and e >= c[3]]

        for idx in reversed(removals):
            del sorted_[idx]

In [24]:
def _remove_subset_l(candidates):
    candidates_ = []
    for l_tag in ltags:

        # Sort by length of L
        sorted_ = sorted(
            filter(lambda x:x[1] == l_tag, candidates), 
            key=lambda x:-(x[3] - x[2])
        )

        while sorted_:
            candidates_.append(sorted_.pop(0))
            (b, e) = (candidates_[-1][2], candidates_[-1][3])

             # Find overlapped
            removals = [i for i, c in enumerate(sorted_) if b <= c[2] and e >= c[3]]

            for idx in reversed(removals):
                del sorted_[idx]
    
    candidates_ = sorted(candidates_, key=lambda x:x[2])
    return candidates_

'아이' 는 단어열 후보에서 제외되었습니다. 

In [25]:
candidates = _initialize_L(sent)
_remove_subset_l(candidates)

[['너무너무너무', '명사', 0, 6],
 ['너무', '부사', 0, 2],
 ['너무', '부사', 2, 4],
 ['너무', '부사', 4, 6],
 ['아이오아이', '명사', 8, 13],
 ['노래', '명사', 14, 16]]

조사는 명사 뒤에 위치합니다. 즉, 명사를 먼저 잘 인식하고, 그 뒤에 위치는 단어가 조사인지 확인해도 됩니다. 이를 위해서 우리는 templates 를 이용합니다. 아래의 템플릿은 명사 뒤에 '조사', '동사', '형용사'가 위치할 수 있다는 의미입니다. 

	templates = {'명사': ('조사', '동사', '형용사')}

앞서 언급한 것처럼 본래 명사 다음에는 조사만 위치할 수 있습니다만, '노래하는'과 같이 동사형전성어미 '-하'를 동사로 생각한다면 '노래/명사 + 하는/동사'로 생각할 수도 있습니다. 그리고 형태소 분석까지 해야 하는 상황일 때에는, 어절의 L + [R] 구조에서 'L/명사 + R/동사' 의 형식일 때, 'R/동사' 를 '동사형전성어미 + 어말어미'로 분해하면 됩니다. 

\_expand\_R(t, candidats) 는 문장과 L 의 단어열 후보를 입력받아, L 에 적합한 R 을 확장합니다. 이 결과물은 L + [R] 구조의 어절이 되기 때문에 LR 이라는 namedtuple 을 이용합니다. 

In [26]:
templates = {'명사': ('조사', '동사', '형용사')}
rtags = {tag for tags in templates.values() for tag in tags}

def _expand_R(t, candidates):
    n = len(t)
    expanded = []

    for (l, l_tag, b, e1) in candidates:
        last = min(dictionary.max_length + e1, n)

        for e2 in range(e1, last + 1):
            r = t[e1:e2]

            if not r:
                expanded.append(LR(l, l_tag, r, None, b, e1, e2))
            else:
                for r_tag in templates.get(l_tag, []):
                    if not dictionary.word_is_tag(r, r_tag):
                        continue
                    expanded.append(LR(l, l_tag, r, r_tag, b, e1, e2))

    return sorted(expanded, key=lambda x:x.b)

L 에 R 이 확장된 LR 후보들이 만들어졌습니다.

In [27]:
_expand_R(sent, candidates)

[LR(l='너무', l_tag='부사', r='', r_tag=None, b=0, m=2, e=2),
 LR(l='너무너무너무', l_tag='명사', r='', r_tag=None, b=0, m=6, e=6),
 LR(l='너무너무너무', l_tag='명사', r='라', r_tag='조사', b=0, m=6, e=7),
 LR(l='너무너무너무', l_tag='명사', r='라는', r_tag='조사', b=0, m=6, e=8),
 LR(l='너무', l_tag='부사', r='', r_tag=None, b=2, m=4, e=4),
 LR(l='너무', l_tag='부사', r='', r_tag=None, b=4, m=6, e=6),
 LR(l='아이', l_tag='명사', r='', r_tag=None, b=8, m=10, e=10),
 LR(l='아이오아이', l_tag='명사', r='', r_tag=None, b=8, m=13, e=13),
 LR(l='아이오아이', l_tag='명사', r='의', r_tag='조사', b=8, m=13, e=14),
 LR(l='아이', l_tag='명사', r='', r_tag=None, b=11, m=13, e=13),
 LR(l='아이', l_tag='명사', r='의', r_tag='조사', b=11, m=13, e=14),
 LR(l='노래', l_tag='명사', r='', r_tag=None, b=14, m=16, e=16),
 LR(l='노래', l_tag='명사', r='입니다', r_tag='조사', b=14, m=16, e=19)]

여기에서도 최장일치법을 적용할 수 있습니다. '-라', '-라는' 이라는 두 개의 조사는 서로 위치가 겹칩니다. 이때에는 같은 품사인 단어 중에서 가장 긴 단어를 선택합니다. 이 과정이 \_remove_subset_r(expanded) 에 구현되어 있습니다. 
 
	LR(l='너무너무너무', l_tag='명사', r='라', r_tag='조사', b=0, m=6, e=7),
	LR(l='너무너무너무', l_tag='명사', r='라는', r_tag='조사', b=0, m=6, e=8),

In [28]:
def _remove_subset_r(expanded):
    expanded_ = []
    rtags = [rtag for rtag in rtags] + [None]
    for r_tag in rtags:

        # Sory by length of R
        sorted_ = sorted(
            filter(lambda x:x.r_tag == r_tag, expanded),
            key=lambda x:-(x.e - x.m)
        )

        while sorted_:
            expanded_.append(sorted_.pop(0))

            l_tag = expanded_[-1].l_tag
            (b, m, e) = expanded_[-1][-3:]

             # Find subset
            removals = [i for i, c in enumerate(sorted_) 
                        if ((c.l_tag == l_tag) 
                            and (b == c.b and m == c.m)
                            and (m <= c.m and e >= c.e))
                       ]

            for idx in reversed(removals):
                del sorted_[idx]

    return expanded_

이 네가지 과정을 모두 모아 LRTemplateMatcher 라는 generator 를 만들겠습니다. 

In [29]:
class LRTemplateMatcher(BaseTemplateMatcher):
    def __init__(self, dictionary, ltags=None, templates=None):
        if not ltags:
            ltags = {'명사', '형용사', '동사', '부사', '감탄사'}
        if not templates:
            templates = {'명사': ('조사', '동사', '형용사')}
        
        self.dictionary = dictionary
        self.ltags = ltags
        self.rtags = {tag for tags in templates.values() for tag in tags}
        self.templates = templates
            
    def generate(self, token):
        candidates = self._initialize_L(token)
        candidates = self._expand_R(token, candidates)
        return candidates
    
    def _pos_L(self, word):
        poses = self.dictionary.get_pos(word)
        poses = {pos for pos in poses if pos in self.ltags}
        return poses

    def _initialize_L(self, t):
        n = len(t)
        candidates = []
        
        for b in range(n):
            for e in range(b+2, min(n, b + self.dictionary.max_length) + 1):
                l = t[b:e]
                l_tags = self._pos_L(l)
                
                if not l_tags:
                    continue
                    
                for l_tag in l_tags:
                    candidates.append([l, l_tag, b, e])
                    
        candidates = self._remove_subset_l(candidates)
        return sorted(candidates, key=lambda x:x[2])

    def _remove_subset_l(self, candidates):
        candidates_ = []
        for l_tag in self.ltags:
            
            # Sort by length of L
            sorted_ = sorted(
                filter(lambda x:x[1] == l_tag, candidates), 
                key=lambda x:-(x[3] - x[2])
            )
            
            while sorted_:
                candidates_.append(sorted_.pop(0))
                (b, e) = (candidates_[-1][2], candidates_[-1][3])
                
                 # Find overlapped
                removals = [i for i, c in enumerate(sorted_) if b <= c[2] and e >= c[3]]
                
                for idx in reversed(removals):
                    del sorted_[idx]
                    
        return candidates_

    def _expand_R(self, t, candidates):
        n = len(t)
        expanded = []

        for (l, l_tag, b, e1) in candidates:
            last = min(self.dictionary.max_length + e1, n)
            
            for e2 in range(e1, last + 1):
                r = t[e1:e2]
                
                if not r:
                    expanded.append(LR(l, l_tag, r, None, b, e1, e2))
                else:
                    for r_tag in self.templates.get(l_tag, []):
                        if not self.dictionary.word_is_tag(r, r_tag):
                            continue
                        expanded.append(LR(l, l_tag, r, r_tag, b, e1, e2))
                        
        expanded = self._remove_subset_r(expanded)
        return sorted(expanded, key=lambda x:x.b)

    def _remove_subset_r(self, expanded):
        expanded_ = []
        rtags = [rtag for rtag in self.rtags] + [None]
        for r_tag in rtags:
            
            # Sory by length of R
            sorted_ = sorted(
                filter(lambda x:x.r_tag == r_tag, expanded),
                key=lambda x:-(x.e - x.m)
            )
            
            while sorted_:
                expanded_.append(sorted_.pop(0))
                
                l_tag = expanded_[-1].l_tag
                (b, m, e) = expanded_[-1][-3:]
                
                 # Find subset
                removals = [i for i, c in enumerate(sorted_) 
                            if ((c.l_tag == l_tag) 
                                and (b == c.b and m == c.m)
                                and (m <= c.m and e >= c.e))
                           ]

                for idx in reversed(removals):
                    del sorted_[idx]

        return expanded_
    
lr_generator = LRTemplateMatcher(dictionary)

In [30]:
lr_generator.generate('아이오아이는')

[LR(l='아이오아이', l_tag='명사', r='는', r_tag='조사', b=0, m=5, e=6),
 LR(l='아이오아이', l_tag='명사', r='', r_tag=None, b=0, m=5, e=5)]

In [31]:
lr_generator.generate(sent.replace(' ', ''))

[LR(l='너무너무너무', l_tag='명사', r='라는', r_tag='조사', b=0, m=6, e=8),
 LR(l='너무너무너무', l_tag='명사', r='', r_tag=None, b=0, m=6, e=6),
 LR(l='너무', l_tag='부사', r='', r_tag=None, b=0, m=2, e=2),
 LR(l='너무', l_tag='부사', r='', r_tag=None, b=2, m=4, e=4),
 LR(l='너무', l_tag='부사', r='', r_tag=None, b=4, m=6, e=6),
 LR(l='아이오아이', l_tag='명사', r='의', r_tag='조사', b=8, m=13, e=14),
 LR(l='아이오아이', l_tag='명사', r='', r_tag=None, b=8, m=13, e=13),
 LR(l='노래', l_tag='명사', r='입니다', r_tag='조사', b=14, m=16, e=19),
 LR(l='노래', l_tag='명사', r='', r_tag=None, b=14, m=16, e=16)]

Evaluator 는 앞선 SimpleEojeolEvaluator 와 비슷합니다. 단, LR 이라는 namedtuple 을 이용하고 있으므로, 이에 맞춰서 evaluate(lr) 이라는 함수를 만들어 줍니다. 

In [32]:
class LREvaluator(BaseEvaluator):
    def __init__(self, weights=None):
        if not weights:
            weights = (
                ('is_noun_phrase', 0.3),
                ('lr_are_known', 0.15),
                ('is_verb', 0.2),
                ('is_adjective', 0.1),
                ('l_len', 0.1),
                ('r_len', 0.1),
                ('l_is_a_syllable', -0.2),
            )
        self.weights = weights

    def select_best(self, candidates):
        scored_candidates = [(c, self.evaluate(c)) for c in candidates]
        best = self._remove_overlapped(scored_candidates)
        return sorted(best, key=lambda x:x[2])
    
    def evaluate(self, lr):
        is_noun_phrase = lr.l_tag == '명사'
        lr_are_known = lr.l_tag is not None and lr.r_tag is not None
        # tag preference
        is_verb = lr.l_tag == '동사'
        is_adjective = lr.l_tag == '형용사'
        # length
        l_len = lr.m - lr.b
        r_len = lr.e - lr.m
        l_is_a_syllable = l_len == 1
        
        score = (is_noun_phrase * self.weights[0][1]
                 + lr_are_known * self.weights[1][1]
                 + is_verb * self.weights[2][1]
                 + is_adjective * self.weights[3][1]
                 + l_len * self.weights[4][1]
                 + r_len * self.weights[5][1]
                 + l_is_a_syllable * self.weights[6][1]
                )

        return score

    def _remove_overlapped(self, scored_candidates):
        best = []
        sorted_ = sorted(scored_candidates, key=lambda x:-x[1])
        
        while sorted_:
            best.append(sorted_.pop(0)[0])
            (b, e) = (best[-1].b, best[-1].e)
            
            # Find overlapped 
            removals = [i for i, (c, _) in enumerate(sorted_) if b < c.e and e > c.b]
            
            for idx in reversed(removals):
                del sorted_[idx]
                
        return sorted(best, key=lambda x:x[2])

lr_evaluator = LREvaluator()

만들어진 LREvaluator 를 테스트해봅니다.

In [34]:
candidates = lr_generator.generate(sent)
pprint(candidates)

[LR(l='너무너무너무', l_tag='명사', r='라는', r_tag='조사', b=0, m=6, e=8),
 LR(l='너무너무너무', l_tag='명사', r='', r_tag=None, b=0, m=6, e=6),
 LR(l='너무', l_tag='부사', r='', r_tag=None, b=0, m=2, e=2),
 LR(l='너무', l_tag='부사', r='', r_tag=None, b=2, m=4, e=4),
 LR(l='너무', l_tag='부사', r='', r_tag=None, b=4, m=6, e=6),
 LR(l='아이오아이', l_tag='명사', r='의', r_tag='조사', b=8, m=13, e=14),
 LR(l='아이오아이', l_tag='명사', r='', r_tag=None, b=8, m=13, e=13),
 LR(l='노래', l_tag='명사', r='입니다', r_tag='조사', b=14, m=16, e=19),
 LR(l='노래', l_tag='명사', r='', r_tag=None, b=14, m=16, e=16)]


이를 이용하여 주어진 문장을 품사 판별한 결과입니다.

In [35]:
pprint(lr_evaluator.select_best(candidates))

[LR(l='너무너무너무', l_tag='명사', r='라는', r_tag='조사', b=0, m=6, e=8),
 LR(l='아이오아이', l_tag='명사', r='의', r_tag='조사', b=8, m=13, e=14),
 LR(l='노래', l_tag='명사', r='입니다', r_tag='조사', b=14, m=16, e=19)]


이 결과를 사용하기 편한 형태 (단어, 품사)열 로 만들고, flatten 기능을 추가하여 SimpleTagger 를 만듭니다. 

In [36]:
class SimpleTagger(BaseTagger):
    def tag(self, sentence, flatten=True):
        sent_ = []
        eojeols = sentence.split()
        for eojeol in eojeols:
            candidates = self.generator.generate(eojeol)
            best = self.evaluator.select_best(candidates)
            
            if self.postprocessor:
                postprocessed = self.postprocessor.postprocess(eojeol, best)
            else:
                postprocessed = best
            
            # Flatten in a token
            postprocessed_ = []
            for word in postprocessed:
                if word.l:
                    postprocessed_.append((word.l, word.l_tag))
                if word.r:
                    postprocessed_.append((word.r, word.r_tag))
            
            sent_.append(postprocessed_)
        
        # Flatten in a sentence
        if flatten:
            sent_ = [word for words in sent_ for word in words]
        return sent_

아직 한가지 빼먹은 부분이 있습니다. 우리는 문장에서 사전에 등록된 단어에 대해서만 품사를 부여하였습니다. 그 결과 아래 문장의 '우리', '!' 는 단어열에서 제외되었습니다. 

In [37]:
lr_tagger = SimpleTagger(lr_generator, lr_evaluator)

sent = '너무너무너무라는우리아이오아이의노래입니다!'
lr_tagger.tag(sent)

[('너무너무너무', '명사'),
 ('라는', '조사'),
 ('아이오아이', '명사'),
 ('의', '조사'),
 ('노래', '명사'),
 ('입니다', '조사')]

우리는 사전 기반의 품사 판별기를 만드는 것이 목적이기 때문에 모르는 단어는 모른다고 솔직하게 태깅을 하겠습니다. LR 에서는 (b, e) 정보가 있습니다. 이전 단어의 e 와 현재 단어의 b 가 같지 않다면 중간에 인식되지 않은 단어가 있다는 의미입니다. 리스트 맨 앞의 LR 의 b 가 0 이 아니라면 이 역시 문장의 앞부분의 단어를 모른다는 의미입니다. 비슷하게, 리스트 맨 뒤의 LR 의 e가 글자 수와 다르다면 뒷부분을 제대로 인식하지 못했다는 의미입니다. LR 리스트의 b, e 를 확인하며 알고리즘이 인식하지 못한 부분에 대하여 None tag 를 부여하여 LR list 를 추가하는 UnknownLRPostprocessor 를 만듭니다. 

In [38]:
class UnknowLRPostprocessor(BasePostprocessor):
    def postprocess(self, token, words):
        n = len(token)
        adds = []
        if words and words[0].b > 0:
            adds.append(self._add_first_subword(token, words))
        if words and words[-1].e < n:
            adds.append(self._add_last_subword(token, words, n))
        adds += self._add_inter_subwords(token, words)
        post = words + adds
        return sorted(post, key=lambda x:x.b)

    def _add_last_subword(self, token, words, n):
        b = words[-1].e
        subword = token[b:]
        return LR(subword, None, '', None, b, n, n)

    def _add_first_subword(self, token, words):    
        e = words[0].b
        subword = token[0:e]
        return LR(subword, None, '', None, 0, e, e)
    
    def _add_inter_subwords(self, token, words):
        adds = []        
        for i, base in enumerate(words[:-1]):
            if base.e == words[i+1].b:
                continue
            b = base.e
            e = words[i+1].b
            subword = token[b:e]
            adds.append(LR(subword, None, '', None, b, e, e))
        return adds

unknown_postprocessor = UnknowLRPostprocessor()

이를 이용하면 '우리', '!' 는 품사 인식을 하지 못한 단어로 출력됩니다. 이 외에도 적절한 후처리 기능을 추가할 수 있습니다. 

In [39]:
lr_tagger = SimpleTagger(lr_generator, lr_evaluator, unknown_postprocessor)
lr_tagger.tag('너무너무너무는우리아이오아이의노래입니다!', )

[('너무너무너무', '명사'),
 ('는', '조사'),
 ('우리', None),
 ('아이오아이', '명사'),
 ('의', '조사'),
 ('노래', '명사'),
 ('입니다', '조사'),
 ('!', None)]

우리는 단어 사전을 이용하는 간단한 품사 판별기를 만들어 보았습니다. 사전 기반 품사 판별기의 핵심은 사전을 잘 구성하는 것입니다. 그러나 데이터를 분석하기도 전에 그 데이터에 등장할 단어들을 알 수는 없습니다. 또한 어떤 단어 사전도 데이터의 모든 단어를 포함하지는 않습니다. 언제나 새로운 개념이 만들어지고, 때로는 오탈자가 발생하여 사전에 없는 단어가 데이터에 입력되기도 합니다. 혹은 대화체에서는 '몰라용' 과 같이 사라은 쉽게 인식되지만 '몰라요' 와는 다른 형태의 단어들이 이용되기도 합니다. 우리는 뒤이어 이러한 문제들을 처리하는 방법에 대하여 알아보겠습니다. 